In [11]:
import huggingface
from datasets import load_dataset
import requests
import json
import os
from dotenv import load_dotenv

load_dotenv()

True

In [5]:
dataset = load_dataset("gsm8k", "main")

README.md:   0%|          | 0.00/7.94k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/2.31M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/419k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/7473 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1319 [00:00<?, ? examples/s]

In [12]:
print(dataset["train"][0])
print(dataset["train"][0]['question'])


{'question': 'Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?', 'answer': 'Natalia sold 48/2 = <<48/2=24>>24 clips in May.\nNatalia sold 48+24 = <<48+24=72>>72 clips altogether in April and May.\n#### 72'}
Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?


In [25]:
print(dataset["train"][0]['answer'])
print(dataset["train"][1]['answer'])
print(dataset["train"][2]['answer'])

Natalia sold 48/2 = <<48/2=24>>24 clips in May.
Natalia sold 48+24 = <<48+24=72>>72 clips altogether in April and May.
#### 72
Weng earns 12/60 = $<<12/60=0.2>>0.2 per minute.
Working 50 minutes, she earned 0.2 x 50 = $<<0.2*50=10>>10.
#### 10
In the beginning, Betty has only 100 / 2 = $<<100/2=50>>50.
Betty's grandparents gave her 15 * 2 = $<<15*2=30>>30.
This means, Betty needs 100 - 50 - 30 - 15 = $<<100-50-30-15=5>>5 more.
#### 5


In [30]:
answers = []
for i in range(len(dataset["train"])):
    sample = dataset["train"][i]
    for line in sample['answer'].split("\n"):
        if line.strip().startswith("####"):
            answers.append(int(line.replace("####", "").strip().replace(",","")))

In [112]:
def get_model_response(prompt, answer, filename):
    try:
        response = requests.post(
            url="https://openrouter.ai/api/v1/chat/completions",
            headers={
                "Authorization": f"Bearer {os.getenv('OPENROUTER_API_KEY')}",
            },
            data=json.dumps({
                "model": "anthropic/claude-3.5-sonnet", 
                "messages": [
                    {
                        "role": "user",
                        "content": (
                            """
                            Solve the math problem provided below. 
                            At the very end of your message, provide the answer to the problem in <ANSWER> </ANSWER> tags:
                            In the answer tags, only provide the answer to the problem, no other text.
                            The answer should be an integer without any other symbols.
                            """
                            + prompt
                        )
                    }
                ]
                
            })
        )
        if response.status_code == 200:
            data = response.json()
            model_response= data['choices'][0]['message']['content']
            model_answer = model_response.split("<ANSWER>")[1].split("</ANSWER>")[0]
            
            print("CORRECT ANSWER: ",answer)
            print("MODEL ANSWER: ",model_answer)
            with open(filename, 'a') as f:
                f.write("QUESTION: " + prompt)
                f.write("MODEL RESPONSE: " + model_response + "\n\n")
                f.write("MODEL ANSWER: " + model_answer + "\n\n")
                f.write("CORRECT ANSWER: " + str(answer) + "\n\n")

            # print(f"Response appended to {filename}")

            return model_answer
        else:
            print(f"Error: {response.status_code}")
    except Exception as e:
        print(f"Error: {e}")

In [ ]:
correct = 0
starting = 100
ending = 200
wrong_indices = []
for i in range(starting,ending):
    answer = get_model_response(dataset["train"][i]['question'])
    answer = int(answer)
    print("CORRECT ANSWER: ",answers[i])
    if (answer == answers[i]):
        correct+=1
    else:
        wrong_indices.append(i)


print("accuracy: ", correct/(ending-starting)) 
print(wrong_indices)

In [64]:
print("INDICES OF PROBLEMS THE MODEL GETS WRONG:")
wrong_indices = [59,63,73, 108, 118, 167]
print(wrong_indices)

INDICES OF PROBLEMS THE MODEL GETS WRONG:
[59, 63, 73, 108, 118, 167]


In [ ]:
for i in wrong_indices:
    answer = get_model_response(dataset["train"][i]['question'])
    answer = int(answer)

    print(dataset["train"][i]['question'])
    print("CORRECT ANSWER: ",answers[i])

    print(dataset["train"][i]['answer'])

## OUTPUTS

MODEL_RESPONSE:  Let me help solve this problem step by step.

1. First, let's identify how many people need trail mix:
   * Baseball team members: 13
   * Coaches: 3
   * Helpers: 2
   * Total people: 13 + 3 + 2 = 18 people

2. Each pack contains 6 pouches of trail mix

3. To find how many packs Roger needs:
   * 18 people ÷ 6 pouches per pack = 3 packs

   However, 3 packs only provides 18 pouches, which is exactly enough. It's usually good to have a few extra, so Roger should buy 4 packs to be safe.

<ANSWER>4</ANSWER>
ANSWER:  4
Response saved to response.txt
CORRECT ANSWER:  3
MODEL_RESPONSE:  Let me help solve this step by step:

MODEL_RESPONSE:  Let me help solve this step by step.

1. First, let's identify Shawna's total goal:
   * Total goal = 30 situps

2. Now, let's count how many situps she's done so far:
   * Monday: 12 situps
   * Tuesday: 19 situps
   * Total done = 12 + 19 = 31 situps

3. To find out how many more she needs to meet her goal:
   * Need = Total goal - Total done
   * Need = 30 - 31
   * Need = -1

Since Shawna has already done 31 situps over Monday and Tuesday, which is more than her goal of 30, she doesn't need to do any additional situps on Wednesday to meet her minimum goal.

<ANSWER>0</ANSWER>
ANSWER:  0
Response saved to response.txt
CORRECT ANSWER:  59
MODEL_RESPONSE:  Let me help solve this step by step.

MODEL_RESPONSE:  Let me help solve this step by step.

1. First, let's identify Shawna's total goal:
   * Total goal = 30 situps

2. Now, let's count how many situps she's done so far:
   * Monday: 12 situps
   * Tuesday: 19 situps
   * Total done = 12 + 19 = 31 situps

3. To find out how many more she needs to meet her goal:
   * Need = Total goal - Total done
   * Need = 30 - 31
   * Need = -1

Since Shawna has already done 31 situps over Monday and Tuesday, which is more than her goal of 30, she doesn't need to do any additional situps on Wednesday to meet her minimum goal.

<ANSWER>0</ANSWER>
ANSWER:  0
Response saved to response.txt
CORRECT ANSWER:  59
MODEL_RESPONSE:  Let me help solve this step by step.

1. Henry took 9 pills per day for 14 days
   * Total pills = 9 × 14 = 126 pills total

2. Out of 9 daily pills:
   * 4 pills cost $1.50 each
   * 5 pills cost $1.50 + $5.50 = $7.00 each

3. Calculate total cost for cheaper pills:
   * Number of cheaper pills = 4 pills × 14 days = 56 pills
   * Cost = 56 × $1.50 = $84.00

4. Calculate total cost for expensive pills:
   * Number of expensive pills = 5 pills × 14 days = 70 pills
   * Cost = 70 × $7.00 = $490.00

5. Total cost for all pills:
   * Total = $84.00 + $490.00 = $574.00

<ANSWER>574</ANSWER>
ANSWER:  574
Response saved to response.txt
CORRECT ANSWER:  41

MODEL_RESPONSE:  Let me help solve this step by step:

1. First hour collection:
   * Joanne collected 15 coins

2. Next two hours:
   * She collected 35 coins

3. Fourth hour:
   * She collected 50 coins
   * She gave 15 coins to her coworker

4. Let's add it all up:
   * First hour: 15 coins
   * Next two hours: 35 coins
   * Fourth hour: 50 coins - 15 coins = 35 coins

5. Total coins = 15 + 35 + 35 = 85 coins

<ANSWER>85</ANSWER>
ANSWER:  85
Response saved to response.txt
CORRECT ANSWER:  120
MODEL_RESPONSE:  Let me help solve this step by step:

## running

In [107]:
train_subset_questions = dataset["train"][100:150]['question']
train_subset_answers = answers[100:150]


print("RUNNING ACCURACY TESTS:")

correct = 0
for i in range(len(train_subset_questions)):
    answer = get_model_response(train_subset_questions[i],train_subset_answers[i])
    answer = int(answer)
    if (answer == train_subset_answers[i]):
        correct+=1

print("ACCURACY for regular: ", correct/len(train_subset_answers))

RUNNING ACCURACY TESTS:
CORRECT ANSWER:  15
MODEL ANSWER:  15
Response appended to response.txt
CORRECT ANSWER:  36
MODEL ANSWER:  36
Response appended to response.txt
CORRECT ANSWER:  25
MODEL ANSWER:  25
Response appended to response.txt
CORRECT ANSWER:  258
MODEL ANSWER:  258
Response appended to response.txt
CORRECT ANSWER:  96
MODEL ANSWER:  96
Response appended to response.txt
CORRECT ANSWER:  320
MODEL ANSWER:  320
Response appended to response.txt
CORRECT ANSWER:  50
MODEL ANSWER:  50
Response appended to response.txt
CORRECT ANSWER:  97
MODEL ANSWER:  97
Response appended to response.txt
CORRECT ANSWER:  41
MODEL ANSWER:  574
Response appended to response.txt
CORRECT ANSWER:  20
MODEL ANSWER:  20
Response appended to response.txt
CORRECT ANSWER:  140
MODEL ANSWER:  140
Response appended to response.txt
CORRECT ANSWER:  6
MODEL ANSWER:  6
Response appended to response.txt
CORRECT ANSWER:  920
MODEL ANSWER:  920
Response appended to response.txt
CORRECT ANSWER:  60
MODEL ANSWER:

In [ ]:
train_subset_questions = dataset["train"][100:105]['question']
train_subset_answers = answers[100:105]

print("RUNNING FAIL@K ACCURACY TESTS:")

correct = 0
for i in range(len(train_subset_questions)):
    passing = True
    print(f"Running the {i}th sample")
    for j in range(4):
        print(f"{j}th try")
        answer = get_model_response(train_subset_questions[i],train_subset_answers[i],"fail.txt")
        answer = int(answer)
        if (answer == train_subset_answers[i]):
            correct+=1
        else:
            passing = False
            print(f"Failed at the {j}th example")
            break
        if passing:
            correct+=1

print("ACCURACY FOR FAIL@K: ", correct/len(train_subset_answers))

RUNNING FAIL@K ACCURACY TESTS:
Running the 0th sample
0th try
CORRECT ANSWER:  15
MODEL ANSWER:  15
1th try
CORRECT ANSWER:  15
MODEL ANSWER:  15
2th try
CORRECT ANSWER:  15
MODEL ANSWER:  15
3th try
CORRECT ANSWER:  15
MODEL ANSWER:  15
Running the 1th sample
0th try
CORRECT ANSWER:  36
MODEL ANSWER:  36
1th try
CORRECT ANSWER:  36
MODEL ANSWER:  36
2th try


KeyboardInterrupt: 

In [113]:
len(train_subset_answers)

5